# Genre Classification 2

In [3]:
import pandas as pd
import numpy as np
import sklearn.preprocessing as pp

In [4]:
#read data from .pkl object file
df = pd.read_pickle('labels_features.pkl')
df.head()

,track_id,zcr_mean,zcr_var,spec_cent_mean,spec_cent_var,spec_roll_mean,spec_roll_var,mfccs_mean,mfccs_var,genre_top
0,2,0.163695,0.008725,3037.644725,9.809958e+05,6214.647549,2.281126e+06,"[-67.3842, 65.155075, -10.669196, 12.176932, -...","[2907.9536, 927.7224, 519.6038, 312.80936, 306...",Hip-Hop
1,5,0.100023,0.004433,2414.124259,7.569450e+05,5437.441882,2.535670e+06,"[-106.58773, 87.23936, 12.156827, 24.140398, -...","[2197.3853, 843.6833, 457.25446, 314.9391, 276...",Hip-Hop
2,10,0.148674,0.000798,2349.559292,1.419561e+05,4315.856035,6.900174e+05,"[-17.238174, 94.44203, -48.614548, 32.370766, ...","[797.04956, 417.18228, 307.51672, 135.75954, 1...",Pop
3,140,0.044322,0.002694,1746.322280,1.049483e+06,3863.175065,5.928252e+06,"[-278.57538, 101.53074, 36.42724, 35.59012, 26...","[7312.72, 1265.0995, 471.97086, 237.86455, 195...",Folk
4,141,0.061836,0.002676,1654.237111,6.403904e+05,3315.637916,2.762245e+06,"[-208.56926, 116.633934, -1.7237115, 26.018599...","[12825.669, 1198.2709, 278.61575, 253.6174, 20...",Folk


## Build Feature Vector

In [5]:
#combine all features into one vector for training
mfccs_mean = np.nan_to_num(np.stack(df['mfccs_mean']))
x = mfccs_mean

mfccs_var = np.nan_to_num(np.stack(df['mfccs_var']))
x = np.append(x,mfccs_var,1)

zcr_mean = np.zeros((len(x),1))
zcr_mean[:,0] = np.array(df['zcr_mean'])
x = np.append(x,zcr_mean,1)

zcr_var = np.zeros((len(x),1))
zcr_var[:,0] = np.array(df['zcr_var'])
x = np.append(x,zcr_var,1)

sc_mean = np.zeros((len(x),1))
sc_mean[:,0] = np.array(df['spec_cent_mean'])
x = np.append(x,sc_mean,1)

sc_var = np.zeros((len(x),1))
sc_var[:,0] = np.array(df['spec_cent_var'])
x = np.append(x,sc_var,1)

sr_mean = np.zeros((len(x),1))
sr_mean[:,0] = np.array(df['spec_roll_mean'])
x = np.append(x,sr_mean,1)

sr_var = np.zeros((len(x),1))
sr_var[:,0] = np.array(df['spec_roll_var'])
x = np.append(x,sr_var,1)

x = pp.scale(x) #scale features to zero mean unit variance

In [6]:
display(mfccs_mean.shape)

(7997, 20)

## Preprocess Labels and Split Train/Test

In [7]:
from sklearn.model_selection import train_test_split
from ast import literal_eval

y_lab = np.array(df['genre_top'],dtype=str) #labels

#encode labels as numbers
le = pp.LabelEncoder()
le.fit(y_lab)
y = le.transform(y_lab)

#train/test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

## ML Models

### Support Vector Machine

In [8]:
from sklearn import svm

clf = svm.SVC()
clf.fit(x_train, y_train)
clf.score(x_test,y_test)

0.5025

### Random Forest

In [9]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(x_train, y_train)
rf.score(x_test,y_test)

0.47125

### K-Nearest Neighbors

In [10]:
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier()
kn.fit(x_train, y_train)
kn.score(x_test, y_test)

0.45

### CNN

In [11]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(x_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])
history = model.fit(x_train, y_train,
                epochs=20,
                batch_size=128)
test_loss, test_acc = model.evaluate(x_test,y_test)
print(test_acc)

Epoch 1/20
50/50 [==============================] - 0s 1ms/step - loss: 1.7669 - accuracy: 0.3755
Epoch 2/20
50/50 [==============================] - 0s 1ms/step - loss: 1.5111 - accuracy: 0.4563
Epoch 3/20
50/50 [==============================] - 0s 1ms/step - loss: 1.4370 - accuracy: 0.4857
Epoch 4/20
50/50 [==============================] - 0s 1ms/step - loss: 1.3842 - accuracy: 0.5048
Epoch 5/20
50/50 [==============================] - 0s 1ms/step - loss: 1.3288 - accuracy: 0.5274
Epoch 6/20
50/50 [==============================] - 0s 1ms/step - loss: 1.2840 - accuracy: 0.5463
Epoch 7/20
50/50 [==============================] - 0s 1ms/step - loss: 1.2392 - accuracy: 0.5599
Epoch 8/20
50/50 [==============================] - 0s 1ms/step - loss: 1.1995 - accuracy: 0.5806
Epoch 9/20
50/50 [==============================] - 0s 1ms/step - loss: 1.1569 - accuracy: 0.6004
Epoch 10/20
50/50 [==============================] - 0s 1ms/step - loss: 1.1175 - accuracy: 0.6098
Epoch 11/20
50/50 [

### CNN Fine-Tune

In [12]:
# Add Early Stop
from keras import models
from keras import layers
from keras.layers import Dropout
from keras import callbacks


model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(x_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])
es_callback = callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(x_train, y_train,callbacks=[es_callback])

test_loss, test_acc = model.evaluate(x_test,y_test)
print(test_acc)

50/50 [==============================] - 0s 583us/step - loss: 1.5945 - accuracy: 0.4406
0.44062501192092896


In [13]:
# Adding Dropout Layer
from keras import models
from keras import layers
from keras.layers import Dropout

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(x_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(layers.Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=20)

test_loss, test_acc = model.evaluate(x_test,y_test)
print(test_acc)

Epoch 1/20
200/200 [==============================] - 0s 873us/step - loss: 1.8955 - accuracy: 0.3120
Epoch 2/20
200/200 [==============================] - 0s 884us/step - loss: 1.6747 - accuracy: 0.4057
Epoch 3/20
200/200 [==============================] - 0s 1ms/step - loss: 1.5955 - accuracy: 0.4361
Epoch 4/20
200/200 [==============================] - 0s 911us/step - loss: 1.5390 - accuracy: 0.4615
Epoch 5/20
200/200 [==============================] - 0s 965us/step - loss: 1.4906 - accuracy: 0.4662
Epoch 6/20
200/200 [==============================] - 0s 906us/step - loss: 1.4765 - accuracy: 0.4840
Epoch 7/20
200/200 [==============================] - 0s 1ms/step - loss: 1.4286 - accuracy: 0.4923
Epoch 8/20
200/200 [==============================] - 0s 956us/step - loss: 1.4003 - accuracy: 0.5079
Epoch 9/20
200/200 [==============================] - 0s 862us/step - loss: 1.3709 - accuracy: 0.5137
Epoch 10/20
200/200 [==============================] - 0s 890us/step - loss: 1.3487 - 

### Testing Combination of Features

In [14]:
# We use Keras to train and predict genre based on different features combination and try to find out which combination works better
from keras import models
from keras import layers
def cal_accuracy_by_selected_features(features):
    y_lab = np.array(df['genre_top'],dtype=str) #labels
    le = pp.LabelEncoder()
    le.fit(y_lab)
    y = le.transform(y_lab)
    X = np.zeros((7997,1))
    
    for feature in features:
        if feature == 'mfccs_mean' or feature == 'mfccs_var': 
            f = np.nan_to_num(np.stack(df[feature]))
            X = np.append(X,f,1)
        else:
            f = np.zeros((len(X),1))
            f[:,0] = np.array(df[feature])
            X = np.append(X,f,1)
    X = pp.scale(X) 

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
    model = models.Sequential()
    model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=128)
    test_loss, test_acc = model.evaluate(X_test,y_test)
 
    return test_acc

In [15]:
import itertools

all_features = ['zcr_mean','zcr_var','spec_cent_mean','spec_cent_var','spec_roll_mean','spec_roll_var','mfccs_mean','mfccs_var']

all_comb = []

for length in range(1, len(all_features)):
    for i in itertools.combinations(all_features, length):
        all_comb += [list(i)]

In [16]:
result = []
for features in all_comb:
    accu = cal_accuracy_by_selected_features(features)
    result += [[features, accu]]

Epoch 1/20
32/32 [==============================] - 0s 1ms/step - loss: 2.2102 - accuracy: 0.1596
Epoch 2/20
32/32 [==============================] - 0s 1ms/step - loss: 2.0634 - accuracy: 0.1851
Epoch 3/20
32/32 [==============================] - 0s 1ms/step - loss: 2.0151 - accuracy: 0.1986
Epoch 4/20
32/32 [==============================] - 0s 1ms/step - loss: 2.0072 - accuracy: 0.2051
Epoch 5/20
32/32 [==============================] - 0s 1ms/step - loss: 2.0056 - accuracy: 0.1981
Epoch 6/20
32/32 [==============================] - 0s 1ms/step - loss: 2.0033 - accuracy: 0.2119
Epoch 7/20
32/32 [==============================] - 0s 1ms/step - loss: 2.0013 - accuracy: 0.2091
Epoch 8/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9982 - accuracy: 0.2036
Epoch 9/20
32/32 [==============================] - 0s 1ms/step - loss: 2.0011 - accuracy: 0.2011
Epoch 10/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9995 - accuracy: 0.2064
Epoch 11/20
32/32 [

125/125 [==============================] - 0s 585us/step - loss: 1.9347 - accuracy: 0.2386
Epoch 1/20
32/32 [==============================] - 0s 1ms/step - loss: 2.1228 - accuracy: 0.1828
Epoch 2/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9638 - accuracy: 0.2236
Epoch 3/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9403 - accuracy: 0.2241
Epoch 4/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9357 - accuracy: 0.2344
Epoch 5/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9272 - accuracy: 0.2389
Epoch 6/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9236 - accuracy: 0.2461
Epoch 7/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9222 - accuracy: 0.2404
Epoch 8/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9177 - accuracy: 0.2424
Epoch 9/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9162 - accuracy: 0.2464
Epoch 10/20
32/32 [========

125/125 [==============================] - 0s 564us/step - loss: 1.8428 - accuracy: 0.3493
Epoch 1/20
32/32 [==============================] - 0s 1ms/step - loss: 2.1503 - accuracy: 0.2014
Epoch 2/20
32/32 [==============================] - 0s 985us/step - loss: 1.9603 - accuracy: 0.2536
Epoch 3/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9169 - accuracy: 0.2671
Epoch 4/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9007 - accuracy: 0.2756
Epoch 5/20
32/32 [==============================] - 0s 998us/step - loss: 1.8892 - accuracy: 0.2746
Epoch 6/20
32/32 [==============================] - 0s 1ms/step - loss: 1.8871 - accuracy: 0.2716
Epoch 7/20
32/32 [==============================] - 0s 1ms/step - loss: 1.8811 - accuracy: 0.2794
Epoch 8/20
32/32 [==============================] - 0s 1ms/step - loss: 1.8787 - accuracy: 0.2744
Epoch 9/20
32/32 [==============================] - 0s 1ms/step - loss: 1.8725 - accuracy: 0.2724
Epoch 10/20
32/32 [====

32/32 [==============================] - 0s 1ms/step - loss: 1.8334 - accuracy: 0.2996
Epoch 20/20
125/125 [==============================] - 0s 530us/step - loss: 1.8322 - accuracy: 0.3116
Epoch 1/20
32/32 [==============================] - 0s 1ms/step - loss: 2.0908 - accuracy: 0.2436
Epoch 2/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9267 - accuracy: 0.2601
Epoch 3/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9016 - accuracy: 0.2636
Epoch 4/20
32/32 [==============================] - 0s 978us/step - loss: 1.8937 - accuracy: 0.2716
Epoch 5/20
32/32 [==============================] - 0s 1ms/step - loss: 1.8873 - accuracy: 0.2844
Epoch 6/20
32/32 [==============================] - 0s 987us/step - loss: 1.8774 - accuracy: 0.2816
Epoch 7/20
32/32 [==============================] - 0s 1ms/step - loss: 1.8734 - accuracy: 0.2854
Epoch 8/20
32/32 [==============================] - 0s 1ms/step - loss: 1.8695 - accuracy: 0.2899
Epoch 9/20
32/32 [====

32/32 [==============================] - 0s 1ms/step - loss: 1.8449 - accuracy: 0.2981
Epoch 19/20
32/32 [==============================] - 0s 1ms/step - loss: 1.8455 - accuracy: 0.2936
Epoch 20/20
125/125 [==============================] - 0s 537us/step - loss: 1.8561 - accuracy: 0.2993
Epoch 1/20
32/32 [==============================] - 0s 1ms/step - loss: 2.1291 - accuracy: 0.2061
Epoch 2/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9617 - accuracy: 0.2291
Epoch 3/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9346 - accuracy: 0.2396
Epoch 4/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9211 - accuracy: 0.2451
Epoch 5/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9098 - accuracy: 0.2481
Epoch 6/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9029 - accuracy: 0.2591
Epoch 7/20
32/32 [==============================] - 0s 1ms/step - loss: 1.9003 - accuracy: 0.2559
Epoch 8/20
32/32 [=======

...

32/32 [==============================] - 0s 2ms/step - loss: 0.8393 - accuracy: 0.7141
Epoch 19/20
32/32 [==============================] - 0s 2ms/step - loss: 0.8085 - accuracy: 0.7246
Epoch 20/20
125/125 [==============================] - 0s 840us/step - loss: 1.7026 - accuracy: 0.4671
Epoch 1/20
32/32 [==============================] - 0s 2ms/step - loss: 1.8830 - accuracy: 0.3229
Epoch 2/20
32/32 [==============================] - 0s 1ms/step - loss: 1.5775 - accuracy: 0.4317
Epoch 3/20
32/32 [==============================] - 0s 1ms/step - loss: 1.4825 - accuracy: 0.4732
Epoch 4/20
32/32 [==============================] - 0s 1ms/step - loss: 1.4317 - accuracy: 0.4895
Epoch 5/20
32/32 [==============================] - 0s 2ms/step - loss: 1.3812 - accuracy: 0.5105
Epoch 6/20
32/32 [==============================] - ETA: 0s - loss: 1.2465 - accuracy: 0.57 - 0s 1ms/step - loss: 1.3244 - accuracy: 0.5305
Epoch 7/20
32/32 [==============================] - 0s 1ms/step - loss: 1.2969 - 

32/32 [==============================] - 0s 1ms/step - loss: 1.3119 - accuracy: 0.5275
Epoch 17/20
32/32 [==============================] - 0s 2ms/step - loss: 1.2814 - accuracy: 0.5485
Epoch 18/20
32/32 [==============================] - 0s 1ms/step - loss: 1.2672 - accuracy: 0.5568
Epoch 19/20
32/32 [==============================] - 0s 1ms/step - loss: 1.2503 - accuracy: 0.5600
Epoch 20/20
125/125 [==============================] - 0s 668us/step - loss: 1.7115 - accuracy: 0.4271
Epoch 1/20
32/32 [==============================] - 0s 2ms/step - loss: 1.8947 - accuracy: 0.3352
Epoch 2/20
32/32 [==============================] - 0s 1ms/step - loss: 1.5825 - accuracy: 0.4337
Epoch 3/20
32/32 [==============================] - 0s 2ms/step - loss: 1.4981 - accuracy: 0.4687
Epoch 4/20
32/32 [==============================] - 0s 2ms/step - loss: 1.4361 - accuracy: 0.4902
Epoch 5/20
32/32 [==============================] - 0s 1ms/step - loss: 1.3839 - accuracy: 0.5078
Epoch 6/20
32/32 [=====

32/32 [==============================] - 0s 1ms/step - loss: 0.9973 - accuracy: 0.6623
Epoch 16/20
32/32 [==============================] - 0s 2ms/step - loss: 0.9599 - accuracy: 0.6753
Epoch 17/20
32/32 [==============================] - 0s 1ms/step - loss: 0.9312 - accuracy: 0.6903
Epoch 18/20
32/32 [==============================] - 0s 1ms/step - loss: 0.8844 - accuracy: 0.7076
Epoch 19/20
32/32 [==============================] - 0s 1ms/step - loss: 0.8468 - accuracy: 0.7246
Epoch 20/20
125/125 [==============================] - 0s 586us/step - loss: 1.6593 - accuracy: 0.4716
Epoch 1/20
32/32 [==============================] - 0s 1ms/step - loss: 1.8454 - accuracy: 0.3379
Epoch 2/20
32/32 [==============================] - 0s 1ms/step - loss: 1.5653 - accuracy: 0.4365
Epoch 3/20
32/32 [==============================] - 0s 2ms/step - loss: 1.4761 - accuracy: 0.4727
Epoch 4/20
32/32 [==============================] - 0s 2ms/step - loss: 1.4193 - accuracy: 0.4942
Epoch 5/20
32/32 [====

32/32 [==============================] - 0s 1ms/step - loss: 1.0102 - accuracy: 0.6553
Epoch 15/20
32/32 [==============================] - 0s 991us/step - loss: 0.9747 - accuracy: 0.6638
Epoch 16/20
32/32 [==============================] - 0s 988us/step - loss: 0.9207 - accuracy: 0.6966
Epoch 17/20
32/32 [==============================] - 0s 1ms/step - loss: 0.8760 - accuracy: 0.7076
Epoch 18/20
32/32 [==============================] - 0s 1ms/step - loss: 0.8467 - accuracy: 0.7226
Epoch 19/20
32/32 [==============================] - 0s 1ms/step - loss: 0.8103 - accuracy: 0.7319
Epoch 20/20
125/125 [==============================] - 0s 553us/step - loss: 1.6922 - accuracy: 0.4791
Epoch 1/20
32/32 [==============================] - 0s 1ms/step - loss: 1.8925 - accuracy: 0.3274
Epoch 2/20
32/32 [==============================] - 0s 1ms/step - loss: 1.5900 - accuracy: 0.4332
Epoch 3/20
32/32 [==============================] - 0s 1ms/step - loss: 1.4978 - accuracy: 0.4632
Epoch 4/20
32/32 

32/32 [==============================] - 0s 1ms/step - loss: 1.0371 - accuracy: 0.6506
Epoch 14/20
32/32 [==============================] - 0s 1ms/step - loss: 0.9992 - accuracy: 0.6578
Epoch 15/20
32/32 [==============================] - 0s 1ms/step - loss: 0.9579 - accuracy: 0.6723
Epoch 16/20
32/32 [==============================] - 0s 1ms/step - loss: 0.9026 - accuracy: 0.7059
Epoch 17/20
32/32 [==============================] - 0s 1ms/step - loss: 0.8673 - accuracy: 0.7119
Epoch 18/20
32/32 [==============================] - 0s 1ms/step - loss: 0.8364 - accuracy: 0.7191
Epoch 19/20
32/32 [==============================] - 0s 1ms/step - loss: 0.7851 - accuracy: 0.7444
Epoch 20/20
125/125 [==============================] - 0s 515us/step - loss: 1.7172 - accuracy: 0.4806
Epoch 1/20
32/32 [==============================] - 0s 2ms/step - loss: 1.9050 - accuracy: 0.3334
Epoch 2/20
32/32 [==============================] - 0s 1ms/step - loss: 1.5829 - accuracy: 0.4335
Epoch 3/20
32/32 [==

In [17]:
result.sort(key=lambda x: -x[1])
for r in result:
    print(r[0])
    print(r[1])

['zcr_mean', 'spec_cent_var', 'spec_roll_mean', 'mfccs_mean', 'mfccs_var']
0.48862215876579285
['zcr_mean', 'spec_cent_mean', 'spec_roll_mean', 'spec_roll_var', 'mfccs_mean', 'mfccs_var']
0.4866216480731964
['spec_cent_mean', 'spec_roll_mean', 'spec_roll_var', 'mfccs_mean', 'mfccs_var']
0.483370840549469
['zcr_mean', 'zcr_var', 'spec_cent_mean', 'mfccs_mean', 'mfccs_var']
0.4823705852031708
['zcr_mean', 'zcr_var', 'spec_cent_var', 'spec_roll_mean', 'mfccs_mean', 'mfccs_var']
0.4816204011440277
['zcr_var', 'spec_cent_mean', 'mfccs_mean', 'mfccs_var']
0.48137032985687256
['zcr_var', 'spec_cent_mean', 'spec_cent_var', 'spec_roll_mean', 'spec_roll_var', 'mfccs_mean', 'mfccs_var']
0.48087021708488464
['zcr_mean', 'spec_cent_mean', 'spec_cent_var', 'spec_roll_mean', 'spec_roll_var', 'mfccs_mean', 'mfccs_var']
0.4806201457977295
['zcr_mean', 'zcr_var', 'spec_roll_mean', 'spec_roll_var', 'mfccs_mean', 'mfccs_var']
0.47911977767944336
['zcr_mean', 'zcr_var', 'spec_cent_mean', 'spec_cent_var', '